<a href="https://colab.research.google.com/github/dscoder001/Summerizer-NLP-/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.30.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!nvidia-smi

Fri May 10 09:18:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
device

'cuda'

In [ ]:
import pandas as pd
df  =  pd.read_csv('/content/sample_dataset.csv')
df.head()

,newspaper,date,headline,article,summary
0,Mint,3/23/2012,Views | Dim sum bonds in tandoori land,In a move that could help them reduce borrowin...,The Reserve Bank of India has permitted Indian...
1,The Times of India,11/23/2003,Hazare and Patkar: Do or die,The complete lack of integrity and commitment ...,India's political landscape is criticized for ...
2,The Times of India,11/22/2020,India successfully tests hypersonic technology...,The Defence Research and Development Organisat...,India successfully tests a key component for h...
3,The Times of India,1/1/2016,New Year bashes for some turn less groovy and ...,NAVI MUMBAI: It’s not all rock and roll or hav...,Many residents in Navi Mumbai opted for low-ke...
4,The Times of India,11/22/2020,Scientists develop AI-powered tool to predict ...,Researchers from the Indian Institute of Techn...,Indian scientists create an AI tool that uses ...


In [ ]:
# prompt: print the dataset in dictionary format only columns

print(df.columns.to_frame().to_dict(orient='records'))


[{0: 'newspaper'}, {0: 'date'}, {0: 'headline '}, {0: 'article'}, {0: 'summary'}]


In [ ]:
print(df["article"][0])

In a move that could help them reduce borrowing costs, the Reserve Bank of India last week allowed Indian firms to raise debt in Chinese renminbi, up to a collective total of $1 billion.

Indian companies with trading and investment links with China can now settle transactions by borrowing directly in renminbi, instead of raising money in US dollar and converting it to the Chinese currency, which would be more expensive.

Trade between India and China has grown at 40% year-on-year and stood at $60 billion last year. China remains India’s largest trading partner, accounting for 9.09% of India’s international trade during 2009-10.

Indian power firms could be the biggest beneficiaries. India is targeting 100,000MW of additional electricity capacity in the next five years and needs an investment of over US$400 billion in the power sector to reduce electricity deficits. India’s present power capacity is 170,000MW. Indian firms can now use this facility to finance power sector projects.

Po

In [ ]:
pipe = pipeline('summarization', model = model_ckpt )

pipe_out = pipe(df['article'][0] )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(pipe_out[0]['summary_text'].replace("<n>", ".\n"))

Indian companies with trading and investment links with China can now settle transactions by borrowing directly in renminbi ..
Trade between India and China has grown at 40% year-on-year and stood at $60 billion last year .


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=500, truncation=True)
        headline_encodings = tokenizer(example_batch['headline '], max_length=124, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'headline_labels': headline_encodings['input_ids']
    }

df_pt = df.apply(convert_examples_to_features, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus', num_train_epochs=4, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset= df_pt,
                  )

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=24, training_loss=2.874058167139689, metrics={'train_runtime': 115.8132, 'train_samples_per_second': 3.419, 'train_steps_per_second': 0.207, 'total_flos': 338310006325248.0, 'train_loss': 2.874058167139689, 'epoch': 3.878787878787879})

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(list(article_batch), max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("\n", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    return metric.compute(predictions=decoded_summaries, references=target_batch)

In [ ]:
from datasets import load_metric

rouge_metric = load_metric('rouge')


<ipython-input-21-2b749678b7a9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL"]



In [ ]:
score = calculate_metric_on_test_ds(
    df, rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'article', column_summary= 'summary',
    )

100%|██████████| 50/50 [02:30<00:00,  3.01s/it]


In [ ]:
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

,rouge1,rouge2,rougeL
pegasus,0.355279,0.167581,0.261803


In [ ]:
model_pegasus.save_pretrained("pegasus-cnn-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [ ]:
tokenizer.save_pretrained("cnn_tokenizer")

('cnn_tokenizer/tokenizer_config.json',
 'cnn_tokenizer/special_tokens_map.json',
 'cnn_tokenizer/spiece.model',
 'cnn_tokenizer/added_tokens.json',
 'cnn_tokenizer/tokenizer.json')

In [ ]:
sample_text = df["article"][0]

reference = df["summary"][0]

print(reference)

The Reserve Bank of India has permitted Indian companies with ties to China to raise debt in Chinese renminbi, allowing transactions to be settled directly in the Chinese currency. This move, aimed at reducing borrowing costs, is particularly beneficial for Indian power firms as the country targets 100,000MW of additional electricity capacity in the next five years, requiring over $400 billion in investment. With India-China trade growing at 40% year-on-year, the facility to borrow in renminbi streamlines transactions and potentially eases financing for crucial power sector projects, despite concerns over the increasing imports of power equipment from China.


In [ ]:



gen_kwargs = {"length_penalty": 1.6, "num_beams":9, "max_length": 200}

pipe = pipeline("summarization", model="pegasus-cnn-model",tokenizer=tokenizer)

In [ ]:
gen_kwargs2 = {"length_penalty": 0.2, "num_beams":9, "max_length": 32}

In [ ]:
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

print("\nModel Headline:")
print(pipe(sample_text, **gen_kwargs2)[0]["summary_text"])

Dialogue:
In a move that could help them reduce borrowing costs, the Reserve Bank of India last week allowed Indian firms to raise debt in Chinese renminbi, up to a collective total of $1 billion.

Indian companies with trading and investment links with China can now settle transactions by borrowing directly in renminbi, instead of raising money in US dollar and converting it to the Chinese currency, which would be more expensive.

Trade between India and China has grown at 40% year-on-year and stood at $60 billion last year. China remains India’s largest trading partner, accounting for 9.09% of India’s international trade during 2009-10.

Indian power firms could be the biggest beneficiaries. India is targeting 100,000MW of additional electricity capacity in the next five years and needs an investment of over US$400 billion in the power sector to reduce electricity deficits. India’s present power capacity is 170,000MW. Indian firms can now use this facility to finance power sector pro